# Overview

ZnTrack provides an easy to use framework for creating and tracking Experiments.
It is build on top of [DVC](https://dvc.org/), a tool for Version Controlling Machine Learning Projects.
For an introduction we highly recommend reading the [DVC Getting Started](https://dvc.org/doc/start).
Besides version controlled data management, DVC provides method for building a dependency graph, tracking parameters, comparing metrics, reducing computational overhead and queueing multiple runs.

**Why do I want to use ZnTrack?**

Whilst DVC provides all this functionality it is designed to be programming language independent.
This can require writing custom python scripts, reading and writing config files and managing depencencies.

ZnTrack is designed to make these steps as easy and well integrated with Python as possible.
In comparison  to the DVC backbone, it is aimed directly at python developeres and therefore allows a highly adapted and optimized interface.

In [1]:
from zntrack import config

# When using ZnTrack we can write our code inside a Jupyter notebook.
# We can make use of this functionality by setting the `nb_name` config as follows:
config.nb_name = "01_Intro.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir
temp_dir = cwd_temp_dir()

TypeError: __init__() got an unexpected keyword argument 'ignore_cleanup_errors'

Working with DVC requires a GIT and DVC repository which we can set up easily:

In [ ]:
!git init
!dvc init

## Nodes
DVC organizes its pipeline in multiple stages.
A stage can be created by inheriting from `zntrack.Node` and implementing a `run()` method.

The `run()` method will later be executed by our pipeline manager (e.g. `dvc repro`).
To make dvc familiar with our new Node we can write the input script with the `write_graph()` command.

Let's start with an example of creating a random integer between 0 and a parameterized maximum value.

In [ ]:
from zntrack import Node, zn
from random import randrange

class RandomNumber(Node):
    number = zn.outs()
    maximum = zn.params()
    
    def __init__(self, maximum=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.maximum = maximum
    
    def run(self):
        self.number = randrange(self.maximum)

We can now call `run_and_save()` to create our random number

In [ ]:
RandomNumber(maximum=512).run_and_save()

or we can use `write_graph()` to create the DVC stage and let DVC run the commands for us.
By default ZnTrack will only build the graph without running it. We can change that by passing `no_exec=True`.
You can also pass other commands like `always_changed=True, external=True` to the `write_graph()` command.

In [ ]:
RandomNumber(maximum=512).write_graph(no_exec=False)

To gain access to the results we can load the Node via the classmethod `load()` and look at the number attribute.

In [ ]:
RandomNumber.load().number

In [ ]:
temp_dir.cleanup()